# Simularea răspândirii gripei în București, utilizând metoda Monte Carlo

Acest proiect folosește o simulare Monte Carlo pentru a modela răspândirea gripei în București în perioada noiembrie-martie. Simularea ia în considerare probabilitatea zilnică de infectare și timpul de recuperare pentru fiecare individ din populație, împărțită pe sectoare.

In [79]:
import numpy as np
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

from itertools import accumulate

## Parametrii Simulării

În cadrul simulării, folosim următorii parametri:

- **Populația**: 10.000 de indivizi
- **Probabilitatea de infectare**: Valoare aleatoare între 3% și 5% pentru fiecare individ
- **Perioada de recuperare**: 7-10 zile pentru fiecare persoană infectată
- **Durata simulării**: 150 de zile (de la 1 noiembrie la 31 martie)
- **Sectoare**: Simularea este detaliată la nivel de sector, considerând 6 sectoare cu populații diferite.

Acești parametri au fost aleși pentru a reflecta dinamica unei epidemii de gripă într-un oraș mare.


In [80]:
# parametri pentru simulare
nr_populatie = 10000
prob_infectare = np.random.uniform(0.03, 0.05, nr_populatie)  # probabilitatea ca un individ sa se infecteze este intre 3-5%
nr_zile_refacere = np.zeros(nr_populatie, dtype=int)         
simulare_zile = 150                                           # nr de zile pentru care se face simularea (1 nov - 31 martie)
nr_sectoare = 6

# Simularea Răspândirii Gripei pe Întreaga Populație 

Acest cod simulează răspândirea gripei într-o populație de mărime `nr_populatie` (populația simulată pentru întreg Bucureștiul) pe o perioadă de timp definită (`simulare_zile`).

1. **Calculul Zilnic al Numărului de Infectați**:
   - `posibil_infectati`: Persoanele care nu sunt infectate (cu zile de recuperare de 0).
   - `sansa_infectare`: Valori aleatoare pentru fiecare persoană, reprezentând șansa de infectare zilnică.
   - `infectati`: Persoanele care devin infectate dacă șansa lor este mai mică decât probabilitatea de infectare.
   - **Zilele de recuperare** sunt setate între 7 și 10 zile pentru cei infectați, iar numărul zilnic de infectați este stocat în `nr_infectati_pe_zi`.
   - Zilele de recuperare sunt decrementate zilnic pentru cei infectați până când redevin susceptibili.

2. **Estimarea Probabilității de Infectare**:
   - Simulăm infecția pentru fiecare individ pe toată durata simulării.
   - **Estimarea finală**: Probabilitatea estimată de infectare este calculată ca raport între numărul de infecții și numărul total de persoane pe toată perioada simulată.

3. **Rezultate**:
   - `nr_infectati_pe_zi` conține numărul zilnic de infectați.
   - `estimare` afișează probabilitatea estimată de infectare.

Această metodă oferă o estimare a probabilității de infectare folosind simularea Monte Carlo.


In [81]:
# simulam raspandirea gripei penrtu toate zilele in populatia data
nr_infectati_pe_zi = []

for zi in range(simulare_zile):
    posibil_infectati = nr_zile_refacere == 0                           # persoanele care nu sunt inca infectate
    sansa_infectare = np.random.rand(nr_populatie) 
    infectati = posibil_infectati & (sansa_infectare < prob_infectare)  # persoane deja infectate
    nr_infectati_pe_zi.append(np.sum(infectati))

    # setam zilele de refacere pentru persoanele infectate (intre 7-10 zile)
    nr_zile_refacere[infectati] = np.random.randint(7, 11, size=np.sum(infectati))

    # decrementam zilele de refacere pentru persoanele care sunt inca infectate
    nr_zile_refacere[nr_zile_refacere > 0] -= 1


print("Nr infectati pe zi: ", nr_infectati_pe_zi)

# estimam probabilitatea de infectare cu ajutorul simularii Monte Carlo
infected_count = 0  

# rulam simularea pentru fiecare zi
for _ in range(simulare_zile):
    for i in range(nr_populatie):                # simulam pentru fiecare individ din populatie
        random_value = np.random.rand()          # generarea unei valori random 
        if random_value < prob_infectare[i]:     # daca valoarea random este mai mica decat probabilitatea de infectare, atunci individul este infectat
            infected_count += 1  

# estimare 
estimare = infected_count / (nr_populatie * simulare_zile)
print(f"Probabilitatea estimata de infectare: {estimare:.6f}")

Nr infectati pe zi:  [424, 391, 362, 356, 317, 332, 312, 292, 315, 267, 306, 301, 344, 303, 313, 289, 296, 308, 302, 303, 346, 279, 312, 296, 314, 286, 342, 318, 298, 300, 339, 317, 305, 307, 312, 271, 314, 299, 319, 298, 311, 296, 328, 294, 305, 311, 304, 276, 318, 278, 316, 304, 319, 341, 305, 312, 271, 328, 272, 305, 299, 306, 314, 319, 295, 301, 306, 301, 279, 299, 310, 312, 306, 293, 303, 322, 321, 319, 308, 293, 299, 304, 315, 297, 314, 309, 279, 323, 299, 308, 325, 310, 283, 327, 312, 313, 280, 328, 318, 324, 335, 290, 312, 312, 318, 339, 290, 294, 304, 314, 275, 327, 303, 302, 302, 354, 308, 310, 293, 357, 281, 309, 296, 290, 304, 295, 317, 306, 312, 284, 315, 323, 295, 309, 307, 306, 324, 307, 308, 292, 315, 294, 329, 322, 271, 296, 315, 299, 312, 325]
Probabilitatea estimata de infectare: 0.040077


## Rezultate și Vizualizare

Rezultatele sunt reprezentate grafic folosind librăria Plotly.
Primul grafic figurează modul în care gripa se modifică în fiecare zi, începând din luna noiembrie, până în ultima zi din martie.

Acest grafic ajută la înțelegerea modului în care gripa se răspândește în oraș.


In [82]:
# plotarea rezultatelor 
zile_pe_luna = [30, 31, 31, 28, 31]
luni = ['Nov', 'Dec', 'Jan', 'Feb', 'Mar'] 
pozitie_luna = [1] + [sum(zile_pe_luna[:i]) + 1 for i in range(1, len(zile_pe_luna))]

# for i, luna in enumerate(luni):
#     print(f"1st {luna} is on day {pozitie_luna[i]}")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(simulare_zile), 
    y=nr_infectati_pe_zi, 
    mode='lines', 
    name='Numar infectati in perioda noiembrie - martie', 
    line=dict(color='MediumPurple', width=2)))

fig.update_layout(title=dict(text='Evoluția gripei în București (noiembrie - martie)', x=0.5),
                   xaxis=dict(title='Zile', tickvals=pozitie_luna, ticktext=luni),
                   yaxis_title='Nr persoane infectate',
                   paper_bgcolor='rgba(0,0,0,0)',
                   font=dict(family='system-ui', size=14, color='white'),
                   plot_bgcolor='Gainsboro')
fig.show()

# Simularea Răspândirii Gripei în Sectoarele Bucureștiului

Simularea modelează răspândirea gripei în București, împărțit pe 6 sectoare, pentru o perioadă de 150 de zile.

1. **Populația și Probabilitatea de Infectare**: Fiecare sector are o populație simulată și o probabilitate de infectare aleatorie între 3% și 5%.
   
2. **Zile de Recuperare și Contorizarea Infecțiilor**: Inițial, toți sunt sănătoși. `nr_zile_refacere_sectoare` contorizează zilele de recuperare, iar `nr_infectati_pe_zi_sectoare` stochează zilnic numărul de infectați pe sector.

3. **Simularea Infecțiilor**: 
   - În fiecare zi, pentru fiecare sector, persoanele susceptibile au o șansă de infectare.
   - Persoanele infectate primesc zile de recuperare între 7 și 10 zile.
   
4. **Actualizarea Stării**: Zilele de recuperare scad zilnic pentru cei infectați, iar la final se afișează numărul zilnic de infectați pe sector.

Acest model oferă o imagine a răspândirii gripei pe durata simulării în funcție de populația și probabilitatea sectorială.


In [83]:
# simularea raspandirii gripei in sectoarele Bucurestiului 
populatie_simulata_sectoare = np.array([1192, 1745, 2244, 1546, 1422, 1849])  # simularea populatiei din fiecare sector
prob_infectare_sectoare = np.random.uniform(0.03, 0.05, nr_sectoare)          # probabilitatea de infectare pentru fiecare sector

nr_zile_refacere_sectoare = [np.zeros(pop, dtype=int) for pop in populatie_simulata_sectoare] 
nr_infectati_pe_zi_sectoare = [[] for _ in range(nr_sectoare)]

for zi in range(simulare_zile):
    for s in range(nr_sectoare): # pentru fiecare sector
        posibil_infectati = nr_zile_refacere_sectoare[s] == 0 
        sansa_infectare = np.random.rand(populatie_simulata_sectoare[s])      # determinam persoanele care pot fi infectate in sectorul respectiv

        # daca sansa de infectare este mai mica decat probabilitatea de infectare, atunci persoana devine infectata
        infectati = posibil_infectati & (sansa_infectare < prob_infectare_sectoare[s])
        nr_infectati_pe_zi_sectoare[s].append(np.sum(infectati))

        # setarea zilelor de refacere pentru persoanele infectate (intre 7-10 zile)
        nr_zile_refacere_sectoare[s][infectati] = np.random.randint(7, 11, size=np.sum(infectati))

        # decrementarea zilelor de refacere pentru persoanele care sunt inca infectate
        nr_zile_refacere_sectoare[s][nr_zile_refacere_sectoare[s] > 0] -= 1

# nr de persoane infectate in fiecare sector, in fiecare zi
for s in range(nr_sectoare):
    print(f"Nr infectati pe zi in sectorul {s+1}: {nr_infectati_pe_zi_sectoare[s]}\n")


Nr infectati pe zi in sectorul 1: [42, 53, 41, 26, 35, 29, 28, 24, 30, 34, 26, 27, 25, 25, 23, 41, 22, 37, 35, 26, 34, 32, 36, 28, 26, 32, 30, 30, 33, 19, 41, 33, 30, 28, 27, 30, 34, 34, 30, 33, 27, 30, 29, 34, 31, 37, 31, 31, 41, 31, 28, 41, 24, 37, 35, 37, 40, 32, 32, 34, 39, 36, 25, 36, 34, 31, 35, 21, 23, 32, 28, 36, 31, 32, 24, 31, 31, 39, 28, 44, 29, 38, 36, 32, 31, 28, 40, 38, 34, 33, 24, 27, 34, 49, 20, 33, 42, 23, 37, 29, 26, 34, 33, 30, 23, 27, 40, 33, 25, 32, 36, 34, 32, 26, 33, 36, 29, 27, 41, 33, 39, 30, 29, 34, 35, 42, 27, 26, 29, 25, 38, 40, 33, 23, 29, 24, 37, 36, 30, 30, 24, 27, 21, 34, 35, 27, 41, 25, 29, 35]

Nr infectati pe zi in sectorul 2: [70, 71, 69, 70, 86, 65, 61, 61, 67, 69, 54, 64, 39, 60, 68, 66, 62, 65, 68, 66, 71, 56, 59, 56, 70, 63, 65, 56, 60, 67, 70, 67, 57, 48, 68, 58, 52, 69, 54, 54, 40, 62, 59, 67, 61, 54, 72, 71, 55, 60, 62, 51, 82, 62, 59, 51, 64, 58, 57, 63, 65, 60, 64, 73, 64, 75, 56, 54, 66, 62, 60, 68, 57, 62, 69, 54, 55, 52, 66, 57, 57, 64, 4

## Rezultate și Vizualizare

Acest cod estimează și vizualizează rata de infectare cu gripă în fiecare sector al Bucureștiului utilizând o hartă coropletică.

In [84]:
# nr total de infectati in fiecare sector, in fiecare zi
infectii_pe_sector_toate_zilele = [sum(nr_infectati_pe_zi_sectoare[s]) for s in range(nr_sectoare)]

# rata de infectare in fiecare sector 
infectie_rate_sectoare = [infectii / populatie_simulata_sectoare[s] for s, infectii in enumerate(infectii_pe_sector_toate_zilele)] 

# label ul pentru fiecare sector asa cum e in GeoJSON
label_sector = ['Sector 1', 'Sector 2', 'Sector 3', 'Sector 4', 'Sector 5', 'Sector 6']

# dataframe cu rata de infectare pentru fiecare sector
data_rate = pd.DataFrame({'Sector': label_sector, 'Infection Rate': infectie_rate_sectoare})
print(data_rate)

# incarcarea fisierului GeoJSON 
geojson_fisier = '/Users/alexandrazamfirescu/Flu_Spread_Simulation/BucurestiSectoare.geo.json'

with open(geojson_fisier) as f:
    geojson = json.load(f)

# reprezentare harta coropletica 
fig = px.choropleth(
    data_frame=data_rate,
    geojson=geojson_fisier,
    locations='Sector',
    featureidkey='properties.tags.name', 
    color='Infection Rate',
    color_continuous_scale='Turbo',
    labels={'Infection Rate': 'Infection Rate'},
    projection='mercator'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title=dict(text='Rata de infectare cu gripa in fiecare sector', x=0.5),
    margin={"r":0, "t":40, "l":0, "b":0}
)

fig.show()


     Sector  Infection Rate
0  Sector 1        4.000000
1  Sector 2        5.311175
2  Sector 3        4.146613
3  Sector 4        4.441785
4  Sector 5        4.585795
5  Sector 6        5.312061
